<a href="https://colab.research.google.com/github/leandrocodes/Paralel-Programming-UFMS-CPPP/blob/master/T2_PP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-_borkbz1
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-_borkbz1
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=469e99d0b049956a1a56fec824bdc40f4c574c9e1c565f96e30680af0807e80d
  Stored in directory: /tmp/pip-ephem-wheel-cache-x75qh55r/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


## Pt.1 - Sequencial 💻

In [9]:
%%cu
#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <unistd.h>
#define N 262144

int main(){
    long int a[N];
    long int b[N];
    long int c[N];
    long int soma;

    double time = 0.0;

    for (long int i = 0; i < N; i++){
        a[i] = 1;   
        b[i] = i;
    }

    clock_t begin = clock();
    for (long int i = 0; i < N; i++){
        c[i] = a[i] * b[i];
        soma += c[i];

    }

    clock_t end = clock();
    time += (double)(end - begin) / CLOCKS_PER_SEC;

    printf("Tempo gasto: %f ms", time*1000);
    //printf("\n %li", c[1024]);
    //printf("\n %li", soma);
}

Tempo gasto: 1.733000 ms


## Pt.2 - Global Memory Only 💻

In [0]:
%%cu
#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <unistd.h>

#include <iostream>
#include <numeric>
#include <cuda.h>

#define B 524888
#define T 1024

static void CheckCudaErrorAux (const char *, unsigned, const char *, cudaError_t);
#define CUDA_CHECK_RETURN(value) CheckCudaErrorAux(__FILE__,__LINE__, #value, value)

__global__ void dot( int *a, int *b, int *c ) {
   
   int index = threadIdx.x + blockIdx.x * blockDim.x;
   c[index] = a[index] * b[index];
    
  __syncthreads();
    
  if( 0 == threadIdx.x && 0== blockIdx.x ) {
    int sum = 0;
    for( int i = 0; i < B; i++ )
    sum += c[i];
    atomicAdd(c, sum);
  }

}


int main( void ) {
  int *a, *b, *c;
  int *dev_a, *dev_b, *dev_c;
  int size = B * sizeof( int );
    
  cudaMalloc( (void**)&dev_a, size );
  cudaMalloc( (void**)&dev_b, size );
  cudaMalloc( (void**)&dev_c, sizeof(int) );
    
  a = (int*)malloc( size );
  b = (int*)malloc( size );
  c = (int*)malloc( sizeof(int) );
    
  for (int i = 0; i < B; i++){
		a[i] = 1 + rand() % B; 
    b[i] = 1;
	}

  cudaMemcpy( dev_a, a, size, cudaMemcpyHostToDevice );
  cudaMemcpy( dev_b, b, size, cudaMemcpyHostToDevice );
  
    cudaEvent_t start, stop;
    CUDA_CHECK_RETURN(cudaEventCreate(&start));
    CUDA_CHECK_RETURN(cudaEventCreate(&stop));
    float gpu_time = 0.0f;

    CUDA_CHECK_RETURN(cudaEventRecord(start, 0)); 
    
    
  dot<<< B/T, T>>>( dev_a, dev_b, dev_c );
    

    CUDA_CHECK_RETURN(cudaEventRecord(stop, 0));
    CUDA_CHECK_RETURN(cudaEventSynchronize(stop)); 
    CUDA_CHECK_RETURN(cudaEventElapsedTime(&gpu_time, start, stop));
    
    CUDA_CHECK_RETURN(cudaDeviceSynchronize());
    
	  printf("Tempo de Execução na GPU: %.6f ms \n\n", gpu_time);  
    /* FIM DO CÓDIGO QUE MENSURA O TEMPO DE EXECUÇÃO*/
    

  cudaMemcpy( c, dev_c, sizeof(int), cudaMemcpyDeviceToHost );
    
  
  printf("%i ",*c);
  
  free( a ); free( b ); free( c );
    
  cudaFree( dev_a );
  cudaFree( dev_b );
  cudaFree( dev_c );
   
  return 0;
}

static void CheckCudaErrorAux (const char *file, unsigned line, const char *statement, cudaError_t err){
	if (err == cudaSuccess) return;
	std::cerr << statement<<" returned " << cudaGetErrorString(err) << "("<<err<< ") at "<<file<<":"<<line << std::endl;
	exit (1);
}

cudaEventSynchronize(stop) returned an illegal memory access was encountered(77) at /tmp/tmp39o3seh_/df448796-1f0a-48a2-8b43-a78d0782d1f0.cu:66



## Pt.3 - Shared Memory 💻

In [0]:
%%cu
#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <unistd.h>

#include <iostream>
#include <numeric>
#include <cuda.h>

#define B 1048576
#define T 1024

static void CheckCudaErrorAux (const char *, unsigned, const char *, cudaError_t);
#define CUDA_CHECK_RETURN(value) CheckCudaErrorAux(__FILE__,__LINE__, #value, value)


__global__ void dot( int *a, int *b, int *c ) {
    
  __shared__ int sharedSum[T];
    
   int index = threadIdx.x + blockIdx.x * blockDim.x;
  sharedSum[threadIdx.x] = a[index] * b[index];
    
  __syncthreads();
    
  if( 0 == threadIdx.x ) {
    int sum = 0;
    for( int i = 0; i < T; i++ )
    sum += sharedSum[i];
    atomicAdd(c, sum);
  }

}


int main( void ) {
  int *a, *b, *c;
  int *dev_a, *dev_b, *dev_c;
  int size = B * sizeof( int );
    
  // allocate device copies of a, b, c
  cudaMalloc( (void**)&dev_a, size );
  cudaMalloc( (void**)&dev_b, size );
  cudaMalloc( (void**)&dev_c, sizeof(int) );
    
  a = (int*)malloc( size );
  b = (int*)malloc( size );
  c = (int*)malloc( sizeof(int) );
    
  for (int i = 0; i < B; i++){
		a[i] = 1 + rand() % B; 
    b[i] = 1;
	}

  cudaMemcpy( dev_a, a, size, cudaMemcpyHostToDevice );
  cudaMemcpy( dev_b, b, size, cudaMemcpyHostToDevice );
  
    cudaEvent_t start, stop;
    CUDA_CHECK_RETURN(cudaEventCreate(&start));
    CUDA_CHECK_RETURN(cudaEventCreate(&stop));
    float gpu_time = 0.0f;

    CUDA_CHECK_RETURN(cudaEventRecord(start, 0)); 
    
  dot<<< B/T, T>>>( dev_a, dev_b, dev_c );
    
    CUDA_CHECK_RETURN(cudaEventRecord(stop, 0));
    CUDA_CHECK_RETURN(cudaEventSynchronize(stop)); 
    CUDA_CHECK_RETURN(cudaEventElapsedTime(&gpu_time, start, stop));
    
    CUDA_CHECK_RETURN(cudaDeviceSynchronize());
    
	  printf("Tempo de Execução na GPU: %.6f ms \n\n", gpu_time);  
    
  cudaMemcpy( c, dev_c, sizeof(int), cudaMemcpyDeviceToHost );
    
  printf("%i ",*c);
    
  free( a ); free( b ); free( c );
    
  cudaFree( dev_a );
  cudaFree( dev_b );
  cudaFree( dev_c );
   
  return 0;
}

static void CheckCudaErrorAux (const char *file, unsigned line, const char *statement, cudaError_t err){
	if (err == cudaSuccess) return;
	std::cerr << statement<<" returned " << cudaGetErrorString(err) << "("<<err<< ") at "<<file<<":"<<line << std::endl;
	exit (1);
}

Tempo de Execução na GPU: 0.891904 ms 

-248774778 


Tempo gasto: 0.028000 ms 
CPU: 1.04858e+06



## Pt.4 - Dot Product 💻

In [0]:
%%cu
#include <vector>
#include <iostream>
#define CUDA_CHECK_RETURN(value) CheckCudaErrorAux(__FILE__,__LINE__, #value, value)
typedef float real_t;
const size_t BLOCK_SIZE = 1024;

__global__ void full_dot( const real_t* v1, const real_t* v2, real_t* out, int N ) {
   
    __shared__ real_t cache[ BLOCK_SIZE ];
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    cache[ threadIdx.x ] = 0.f;
   
    while( i < N ) {
        cache[ threadIdx.x ] += v1[ i ] * v2[ i ];
        i += gridDim.x * blockDim.x;
    }
   
    __syncthreads();
   
    i = BLOCK_SIZE / 2;
   
    while( i > 0 ) {
        if( threadIdx.x < i ) cache[ threadIdx.x ] += cache[ threadIdx.x + i ];
        __syncthreads();
        i /= 2;
    }
    
    if( threadIdx.x == 0 ) atomicAdd( out, cache[ 0 ] );
    if( threadIdx.x == 0 ) *out += cache[ 0 ];              
}

real_t dot( const real_t* v1, const real_t* v2, int N ) {
    real_t s = 0;
    for( int i = 0; i != N; ++i ) {
        s += v1[ i ] * v2[ i ];
    }
    return s;
}

__global__ void init_vector( real_t* v, int N ) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    while( i < N ) {
        v[ i ] = 1.0f;//real_t( i ) / 1000000.f;
        i += gridDim.x * blockDim.x;
    } 
}

//------------------------------------------------------------------------------
int main(int argc, char** argv ) {
    
    const size_t ARRAY_SIZE = 1048576;
    const int BLOCKS = 1024;
    const int THREADS_PER_BLOCK = BLOCK_SIZE;                                  
    const size_t SIZE = ARRAY_SIZE * sizeof( real_t );
    
    // device storage
    real_t* dev_v1 = 0;  // vector 1
    real_t* dev_v2 = 0;  // vector 2
    real_t* dev_out = 0; // result
    cudaMalloc( &dev_v1,  SIZE );
    cudaMalloc( &dev_v2,  SIZE );
    cudaMalloc( &dev_out, sizeof( real_t ) );

    // host storage
    std::vector< real_t > host_v1( ARRAY_SIZE );
    std::vector< real_t > host_v2( ARRAY_SIZE );
    real_t host_out = 0.f;


    // initialize vector 1 with kernel; much faster than using for loops on the cpu
    init_vector<<< 1048576, 1024  >>>( dev_v1, ARRAY_SIZE );
    cudaMemcpy( &host_v1[ 0 ], dev_v1, SIZE, cudaMemcpyDeviceToHost );
    
    // initialize vector 2 with kernel; much faster than using for loops on the cpu
    init_vector<<< 1048576, 1024  >>>( dev_v2, ARRAY_SIZE );
    cudaMemcpy( &host_v2[ 0 ], dev_v2, SIZE, cudaMemcpyDeviceToHost );
    
    
    cudaMemset( dev_out, 0, sizeof( real_t) );   
    
    double time = 0.0;
    clock_t begin = clock();
    // execute kernel
    full_dot<<<BLOCKS, THREADS_PER_BLOCK>>>( dev_v1, dev_v2, dev_out, ARRAY_SIZE );
    clock_t end = clock();
    time += (double)(end - begin) / CLOCKS_PER_SEC;
    printf("Tempo gasto: %f ms \n", time*1000);

    // copy output data from device(gpu) to host(cpu)
    cudaMemcpy( &host_out, dev_out, sizeof( real_t ), cudaMemcpyDeviceToHost );

    // print dot product by summing up the partially reduced vectors
    // std::cout << "GPU: " << host_out << std::endl;    

    // print dot product on cpu
    std::cout << "CPU: " << dot( &host_v1[ 0 ], &host_v2[ 0 ], ARRAY_SIZE ) << std::endl;

    // free memory
    cudaFree( dev_v1 );
    cudaFree( dev_v2 );
    cudaFree( dev_out );

    return 0;
}